## Notebook Start

### Load Packages

In [1]:
import requests
import json
import pandas as pd
from collections import defaultdict
from collections import Counter
import argparse

In [2]:
api_url = "https://uclabruins.com/api/v2/EventsResults/results?sportId=9&$pageIndex=0&$pageSize=50"
response = requests.get(api_url)


In [15]:
data = response.json()
data["items"][0]

{'id': 34034,
 'gamePregameStoryId': 69064,
 'gamePostgameStoryId': 69072,
 'date': '2025-05-16T17:00:00',
 'endDate': None,
 'dateUtc': '2025-05-17T00:00:00.0000001Z',
 'endDateUtc': None,
 'time': '5:00 PM',
 'isDoubleHeader': False,
 'showAtVs': True,
 'tbd': False,
 'allDay': False,
 'teamPrefix': None,
 'status': 'O',
 'locationIndicator': 'N',
 'neutralHometeam': False,
 'location': 'Waco, Texas',
 'isConference': False,
 'conference': 'Big Ten',
 'conferenceAbbrev': 'Big Ten',
 'conferenceLogo': None,
 'isSpotlight': False,
 'type': 'recent',
 'tournament': "NCAA Men's Tennis Championship",
 'ticketLink': None,
 'sport': {'nonSport': False,
  'atVs': True,
  'gameSynonym': 'Match',
  'globalSportShortName': 'mten',
  'shortTitle': '',
  'scheduleId': 2675,
  'abbrev': 'MTEN',
  'shortDisplay': None,
  'shortName': 'mten',
  'globalSportId': None,
  'globalSportNameSlug': None,
  'globalSportGender': 'm',
  'id': 9,
  'title': "Men's Tennis"},
 'schedule': {'id': 2675,
  'title':

In [67]:
data["items"][0].get("result")

{'status': 'L',
 'teamScore': '1',
 'opponentScore': '4',
 'preScore': None,
 'postScore': None,
 'boxScore': '/boxscore.aspx?id=34034',
 'gameId': 34034,
 'lineScores': {'game_winner': 'H',
  'this_team_is_home_team': False,
  'home_full_name': 'Texas',
  'home_short_name': 'TEX',
  'away_full_name': 'UCLA',
  'away_short_name': 'UCLA',
  'period_label': None,
  'periods': ['F'],
  'period_home_score': ['4'],
  'period_away_score': ['1']}}

In [92]:
bsurllist = []
box_score_api_urls = []

if response.status_code == 200:
    data = response.json()
    all_games = []

    for game in data.get("items", []):
        game_data = {}
        game_data["date"] = game.get("date", "No date available").split("T")[0]
        game_data["location"] = game.get("location", "No location available")
        
        opponent_title = game.get("opponent", {}).get("title", "")
        opponent_parts = opponent_title.split()

        # Only assign seed if it exists
        seed = opponent_parts[1] if len(opponent_parts) > 1 else None
        game_data["opponent_seed"] = seed if seed and seed.isdigit() else None

        game_data["isConference"] = game.get("isConference")
        game_data["tournament"] = game.get("tournament")
        
        result = game.get("result", {}) or {}
        boxScore = result.get("boxScore")
        if isinstance(boxScore, str) and boxScore.startswith("/"):
            boxScore = "https://uclabruins.com/api/v2/Stats/boxscore/" + boxScore.split("=")[-1]
        game_data["boxScore"] = boxScore

        game_data["overallScore"] = str(result.get("teamScore")) + '-' + str(result.get("opponentScore"))

        lineScores = result.get("lineScores", {}) or {}

        winner_code = lineScores.get("game_winner")
        winner_label = {'H': 'home', 'A': 'away'}.get(winner_code, None)

        game_data["game_winner_label"] = winner_label   

        game_data["this_team_is_home_team"] = lineScores.get("this_team_is_home_team")
        game_data["home_full_name"] = lineScores.get("home_full_name")
        game_data["away_full_name"] = lineScores.get("away_full_name")
        
        facility = game.get("gameFacility", {}) or {}
        game_data["facility_id"] = facility.get("id")
        game_data["facility_location"] = facility.get("title")

        all_games.append(game_data)

    df = pd.DataFrame(all_games)
else:
    print(f"Error: Unable to fetch data (Status Code: {response.status_code})")


In [93]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(25)

date             location opponent_seed  isConference  \
0   2025-05-16          Waco, Texas             3         False   
1   2025-05-09  Los Angeles, Calif.          None         False   
2   2025-05-03     Berkeley, Calif.            11         False   
3   2025-05-02     Berkeley, Calif.          None         False   
4   2025-04-27       Columbus, Ohio             1         False   
5   2025-04-26       Columbus, Ohio             3         False   
6   2025-04-25       Columbus, Ohio             7         False   
7   2025-04-20        Lincoln, Neb.            41          True   
8   2025-04-18        Madison, Wis.            58          True   
9   2025-04-13  Los Angeles, Calif.            28          True   
10  2025-04-11  Los Angeles, Calif.            21          True   
11  2025-04-06   State College, Pa.          None          True   
12  2025-04-04       Columbus, Ohio             3          True   
13  2025-04-01  Los Angeles, Calif.            30          True   
14  2025-03-30  Los Angeles, Calif.            47          True   
15  2025-03-29  Los Angeles, Calif.            58          True   
16  2025-03-23       Evanston, Ill.            60          True   
17  2025-03-21         Urbana, Ill.            23          True   
18  2025-03-09  Los Angeles, Calif.            44          True   
19  2025-03-07  Los Angeles, Calif.            38          True   
20  2025-02-28     La Jolla, Calif.          None         False   
21  2025-02-22  Los Angeles, Calif.            25         False   
22  2025-02-16  Los Angeles, Calif.            25         False   
23  2025-02-02  Los Angeles, Calif.            13         False   
24  2025-02-01  Los Angeles, Calif.          None         False   

                            tournament  \
0       NCAA Men's Tennis Championship   
1            NCAA Super-Regional Round   
2         NCAA First and Second Rounds   
3         NCAA First and Second Rounds   
4      Big Ten Men's Tennis Tournament   
5      Big Ten Men's Tennis Tournament   
6      Big Ten Men's Tennis Tournament   
7                                 None   
8                                 None   
9                                 None   
10                                None   
11                                None   
12                                None   
13                                None   
14                                None   
15                                None   
16                                None   
17                                None   
18                                None   
19                                None   
20  Pacific Coast Doubles Championship   
21                                None   
22                                None   
23                                None   
24                                None   

                                             boxScore overallScore  \
0   https://uclabruins.com/api/v2/Stats/boxscore/3...          1-4   
1   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-2   
2   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-2   
3   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-0   
4   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-3   
5   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-0   
6   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-1   
7   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-1   
8   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-0   
9   https://uclabruins.com/api/v2/Stats/boxscore/3...          4-1   
10  https://uclabruins.com/api/v2/Stats/boxscore/3...          5-0   
11  https://uclabruins.com/api/v2/Stats/boxscore/3...          4-0   
12  https://uclabruins.com/api/v2/Stats/boxscore/3...          0-4   
13  https://uclabruins.com/api/v2/Stats/boxscore/3...          3-4   
14  https://uclabruins.com/api/v2/Stats/boxscore/3...          4-0   
15  https://uclabruins.com/api/v2/Stats/boxscore/3... 